In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.master("local[*]").appName("skew_salt").getOrCreate()

In [2]:
spark

In [4]:
#pip install pandas

In [3]:
import pandas as pd
import numpy as np

In [4]:
# set smaller number of partitions so they can fit the screen
# sc.conf.set('spark.sql.shuffle.partitions', 8)
# disable broadcast join to see the shuffle
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

In [5]:
length = 100
names = np.random.choice(['Bob', 'James', 'Marek', 'Johannes', None], length)
amounts = np.random.randint(0, 1000000, length)

In [8]:
print(names)

['James' 'James' 'Johannes' 'Marek' None None 'Bob' 'Johannes' None
 'James' 'Bob' 'Bob' None None 'Marek' 'Marek' 'Marek' None 'Bob' 'Marek'
 'Bob' None 'Bob' 'James' None None 'Johannes' 'Marek' 'Marek' 'Bob' None
 'Marek' None 'Bob' 'Johannes' 'James' 'James' None 'Marek' 'James'
 'Marek' 'James' 'Marek' 'James' 'Johannes' 'James' None 'James' 'Bob'
 'James' 'Marek' 'Marek' 'Johannes' 'James' None 'Bob' 'James' 'Bob'
 'James' 'Bob' 'Johannes' None None 'James' None None 'Johannes'
 'Johannes' 'Marek' 'Bob' 'Bob' 'Johannes' 'Bob' 'James' 'Marek' 'Marek'
 'Bob' 'James' None 'James' 'Marek' None 'Bob' 'Johannes' None 'Bob' None
 'Marek' 'Johannes' 'Bob' 'Johannes' None 'Marek' 'Bob' 'James' None
 'James' 'James' None 'James']


In [9]:
print(amounts)

[415523 526016 329009 741505 729006 763874 101315 345270 477497 515888
 449283 444624  42344  36856  22324 933766 270533 711415 808580 896006
 398079 578318 440501 232889 786761 255736 755368 710084 751007 824194
 332865 323453 267510 985654 616481 901723 163663 345692 539667  38124
 634412  50019 892369  71741 565325 554397 261314 728657  33462 661247
 972108 862378  58600 799678 591453  68644 470973 320570 170025 881181
 239856 226658 308870  86050 499837 647465 440730 752870 683358 534478
 565361 430192 832993 907432 911245  45338 357045 218894 356328 860536
 184344 773296 303027 413171 492678 494122  21275 156342 600580 486393
 159647 680783 406734 116453 904879 814014 926786 654790 472809 840631]


In [6]:
# generate skewed data
country = np.random.choice(
	['United Kingdom', 'Poland', 'USA', 'Germany', 'Russia'],
	length,
	p = [0.05, 0.05, 0.8, 0.05, 0.05]
)

In [11]:
country

array(['USA', 'Germany', 'USA', 'Russia', 'Poland', 'Germany', 'USA',
       'USA', 'Russia', 'Russia', 'USA', 'USA', 'USA', 'USA', 'Poland',
       'USA', 'USA', 'USA', 'USA', 'USA', 'USA', 'Germany', 'Germany',
       'Poland', 'USA', 'USA', 'USA', 'United Kingdom', 'USA', 'USA',
       'USA', 'Poland', 'Poland', 'USA', 'Poland', 'Poland', 'USA', 'USA',
       'USA', 'USA', 'Poland', 'USA', 'Russia', 'USA', 'Poland', 'USA',
       'Poland', 'USA', 'USA', 'USA', 'USA', 'USA', 'USA', 'USA',
       'United Kingdom', 'USA', 'USA', 'USA', 'USA', 'United Kingdom',
       'USA', 'USA', 'United Kingdom', 'USA', 'USA', 'USA', 'USA', 'USA',
       'USA', 'USA', 'USA', 'USA', 'USA', 'USA', 'United Kingdom', 'USA',
       'USA', 'USA', 'USA', 'United Kingdom', 'USA', 'Russia', 'USA',
       'USA', 'USA', 'USA', 'USA', 'Germany', 'USA', 'USA', 'USA', 'USA',
       'USA', 'Poland', 'USA', 'USA', 'Poland', 'USA', 'USA', 'USA'],
      dtype='<U14')

In [12]:
value,counts = np.unique(country,return_counts = True)
dict(zip(value,counts))

{'Germany': 5, 'Poland': 12, 'Russia': 5, 'USA': 72, 'United Kingdom': 6}

In [7]:
#spark.conf.set("spark.sql.execution.arrow.enabled","true")
data = pd.DataFrame({'name': names, 'amount': amounts, 'country': country})
#transactions = spark.createDataFrame(data).repartition('country')
#data = [{'name': names, 'amount': amounts, 'country': country}]
df = spark.createDataFrame(data)

C:\pyspark\spark-3.3.0-bin-hadoop3\python\pyspark\sql\pandas\conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
C:\pyspark\spark-3.3.0-bin-hadoop3\python\pyspark\sql\pandas\conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [8]:
df.count()

100

In [19]:
countries = spark.createDataFrame(pd.DataFrame({
	'id': [11, 12, 13, 14, 15],
	'country': ['United Kingdom', 'Poland', 'USA', 'Germany', 'Russia']
}))

In [20]:
countries.count()

5

In [21]:
df.groupBy(spark_partition_id()).count().orderBy(desc("count")).show()

+--------------------+-----+
|SPARK_PARTITION_ID()|count|
+--------------------+-----+
|                   7|   16|
|                   0|   12|
|                   2|   12|
|                   1|   12|
|                   6|   12|
|                   3|   12|
|                   4|   12|
|                   5|   12|
+--------------------+-----+



In [22]:
df_join = df.join(countries, 'country')

In [23]:
# check the partitions data
for i, part in enumerate(df_join.rdd.glom().collect()):
	print({i: part})

{0: [Row(country='Germany', name='James', amount=526016, id=14), Row(country='Germany', name=None, amount=763874, id=14), Row(country='Germany', name=None, amount=578318, id=14), Row(country='Germany', name='Bob', amount=440501, id=14), Row(country='Germany', name='Marek', amount=156342, id=14), Row(country='Poland', name=None, amount=729006, id=12), Row(country='Poland', name='Marek', amount=22324, id=12), Row(country='Poland', name='James', amount=232889, id=12), Row(country='Poland', name='Marek', amount=323453, id=12), Row(country='Poland', name=None, amount=267510, id=12), Row(country='Poland', name='Johannes', amount=616481, id=12), Row(country='Poland', name='James', amount=901723, id=12), Row(country='Poland', name='Marek', amount=634412, id=12), Row(country='Poland', name='Johannes', amount=565325, id=12), Row(country='Poland', name=None, amount=261314, id=12), Row(country='Poland', name='Bob', amount=116453, id=12), Row(country='Poland', name='James', amount=926786, id=12), R

In [25]:
df_join.show()

+-------+--------+------+---+
|country|    name|amount| id|
+-------+--------+------+---+
|Germany|   James|526016| 14|
|Germany|    null|763874| 14|
|Germany|    null|578318| 14|
|Germany|     Bob|440501| 14|
|Germany|   Marek|156342| 14|
| Poland|    null|729006| 12|
| Poland|   Marek| 22324| 12|
| Poland|   James|232889| 12|
| Poland|   Marek|323453| 12|
| Poland|    null|267510| 12|
| Poland|Johannes|616481| 12|
| Poland|   James|901723| 12|
| Poland|   Marek|634412| 12|
| Poland|Johannes|565325| 12|
| Poland|    null|261314| 12|
| Poland|     Bob|116453| 12|
| Poland|   James|926786| 12|
| Russia|   Marek|741505| 15|
| Russia|    null|477497| 15|
| Russia|   James|515888| 15|
+-------+--------+------+---+
only showing top 20 rows



In [32]:
df_broadcast = transactions.join(broadcast(countries), 'country')
df_broadcast.show(3)

+-------+--------+------+---+
|country|    name|amount| id|
+-------+--------+------+---+
| Russia|   James|622217| 15|
|    USA|Johannes|567364| 13|
|    USA|   James|196858| 13|
+-------+--------+------+---+
only showing top 3 rows



In [46]:
salt = np.random.randint(1, 200 - 1)
salt

45

In [44]:
print(df.rdd.getNumPartitions())

1


In [48]:
salted_countries = countries.withColumn(
    'salt',
    explode(array([lit(i) for i in range(salt)])))

In [50]:
salted_countries.show(20)

+---+--------------+----+
| id|       country|salt|
+---+--------------+----+
| 11|United Kingdom|   0|
| 11|United Kingdom|   1|
| 11|United Kingdom|   2|
| 11|United Kingdom|   3|
| 11|United Kingdom|   4|
| 11|United Kingdom|   5|
| 11|United Kingdom|   6|
| 11|United Kingdom|   7|
| 11|United Kingdom|   8|
| 11|United Kingdom|   9|
| 11|United Kingdom|  10|
| 11|United Kingdom|  11|
| 11|United Kingdom|  12|
| 11|United Kingdom|  13|
| 11|United Kingdom|  14|
| 11|United Kingdom|  15|
| 11|United Kingdom|  16|
| 11|United Kingdom|  17|
| 11|United Kingdom|  18|
| 11|United Kingdom|  19|
+---+--------------+----+
only showing top 20 rows



In [51]:
salted_transactions = transactions.withColumn('salt', round(rand() * salt))

In [52]:
salted_transactions.show()

+--------+------+-------+----+
|    name|amount|country|salt|
+--------+------+-------+----+
|   James|622217| Russia|15.0|
|Johannes|567364|    USA| 5.0|
|   James|196858|    USA| 3.0|
|   James|333152|    USA|27.0|
|   Marek|728314|    USA|28.0|
|   James|471889|    USA|16.0|
|Johannes|960402|    USA|10.0|
|     Bob|486396|    USA|13.0|
|     Bob|736981|    USA|34.0|
|Johannes|375164|    USA|27.0|
|   Marek| 12631|    USA|27.0|
|    null|688780| Poland|23.0|
|   Marek|125713| Russia|32.0|
|   Marek|318116|Germany|12.0|
|    null|179841|Germany|38.0|
|    null|482374|    USA| 2.0|
|Johannes|263538|    USA|14.0|
|   James|926235|    USA| 6.0|
|    null|531462|    USA|17.0|
|Johannes|777230|    USA|12.0|
+--------+------+-------+----+
only showing top 20 rows



In [54]:
df2 = salted_transactions.join(salted_countries, ['country', 'salt'] ).drop('salt')

In [55]:
df2.show()

+-------+--------+------+---+
|country|    name|amount| id|
+-------+--------+------+---+
|Germany|   Marek|318116| 14|
|Germany|   Marek|453900| 14|
|Germany|    null|179841| 14|
|Germany|Johannes|  3456| 14|
| Poland|Johannes|922173| 12|
| Poland|    null|108694| 12|
| Poland|     Bob|509229| 12|
| Poland|    null|688780| 12|
| Poland|Johannes|402074| 12|
| Russia|   James|622217| 15|
| Russia|Johannes|624933| 15|
| Russia|   Marek|714491| 15|
| Russia|   Marek|125713| 15|
|    USA|     Bob| 75298| 13|
|    USA|    null|482374| 13|
|    USA|   Marek|953144| 13|
|    USA|   Marek|611482| 13|
|    USA|   James|196858| 13|
|    USA|     Bob|247591| 13|
|    USA|     Bob|752279| 13|
+-------+--------+------+---+
only showing top 20 rows

